In [ ]:
%matplotlib inline

import re
import copy
import sys
from enum import Enum
import json
import matplotlib.pyplot as plt
import time
import numpy as np
import random
import plotly.figure_factory as ff
import statistics

In [ ]:

def SPT(possible_tasks):
    # Prendre la tâche qui fini le plus tôt
    smallest_duration = sys.maxsize 
    best_task = None
    for t in possible_tasks:
        if t.duration < smallest_duration:
            smallest_duration = t.duration
            best_task = t
    return best_task

def LPT(possible_tasks):
    # Prendre la tâche qui fini le plus tard
    smallest_duration = 0
    best_task = None
    for t in possible_tasks:
        if t.duration > smallest_duration:
            smallest_duration = t.duration
            best_task = t
    return best_task

class STATE(Enum):
    # Etat possible d'une tâche
    NOT_DONE = '--'
    ON_GOING = '+/-' 
    DONE = '++'

class Task:  
    # Classe définissant une tâche
    
    def __init__(self,ressource,duration,job_num):
        self.ressource = ressource
        self.job_num = job_num
        self.duration = duration
        self.start_time = None
        self.possible_start_time = 0
        
    def reset(self):
        # Remise à zéro des variables de la tâche
        self.start_time = None
        self.possible_start_time = 0
        
    def to_string(self):
        return str(self.ressource) + " " + str(self.duration)
        
    def start(self):
        # Démarre une tâche au plus tôt
        self.start_time = self.possible_start_time
                
    def update_possible_start_time(self,time):
        # Met à jour la date de démarrage possible
        if(self.possible_start_time < time):
            self.possible_start_time = time
    
class TaskForHeuristic(Task):
    # Classe définissant un tâche pour l'heuristique
    
    def __init__(self,ressource,duration,job_num):
        super().__init__(ressource,duration,job_num)
        self.state = STATE.NOT_DONE
        
    def start(self,time):
        # Démarrage de la tâche
        self.start_time = time
        self.state = STATE.ON_GOING
        
    def update(self,time):
        # Mise à jour de la tâche lorsque le temps avance
        if(self.state == STATE.ON_GOING):
            if time >= self.start_time + self.duration:
                self.state = STATE.DONE
                return self.ressource
        return -1
                
    def is_done(self):
        # Renvoie true si la tâche est terminée
        return self.state == STATE.DONE
    
    def casted_task(self):
        # Cast la tâche heuristique en tâche
        return Task(self.ressource,self.duration, self.job_num)
        
class Job:
    # Objet définissant un job
    
    def __init__(self,job_num):
        self.tasks = []
        self.num = job_num
        self.nb_task = 0
        
    def add_task(self,task):
        # Rajoute une tâche au job
        self.tasks.append(task)
        self.nb_task += 1
        
    def cast_task(self):
        # Transforme les tâche d'heuristique en tâche
        for t in self.tasks:
            t = t.casted_task()
        
    def reset(self):
        # Remet à zero l'état du job
        for t in self.tasks:
            t.reset()
        
    def init_possible_start(self):
        # Initialise les temps possible de démarrage des tâches
        delay = 0
        for t in self.tasks:
            t.possible_start= delay
            delay = delay + t.duration
        
    def to_string(self):
        res = "|"
        for t in self.tasks:
            res = res + " " + '{:^6}'.format(t.to_string()) + " |" 
        return res
    
    def start(self, ressources_availability_array, jobs_availability_array, task_ind):
        # Démarre la tâche task_ind au plus tôt possible
        t = self.tasks[task_ind]
        # Démarrage dès que le job ET la ressource sont disponibles
        min_start_time = max(ressources_availability_array[t.ressource],jobs_availability_array[self.num])
        t.start_time = min_start_time
        return t.duration+t.start_time , t.ressource
    
    def update_possible_time_by_ressource(self,ressource,time):
        # Mets à jour les temps possibles de démarrage en fonction de la ressource utilisé et la date de fin
        for t in self.tasks:
            if(t.ressource == ressource):
                t.update_possible_start_time(time)
    
    def get_end_time(self):
        # Renvoie la date de fin du job
        return self.tasks[-1].start_time + self.tasks[-1].duration
        
class JobForHeuristic(Job):
    # Classe définissant un job pour l'heuristique
    def __init__(self,job_num):
        super().__init__(job_num)
        
    def get_next_task(self):
        # Renvoie la prochaine tâche à exécuté si aucune tâche n'est déjà en cours
        on_going = [task for task in self.tasks if task.state == STATE.ON_GOING]
        if  len(on_going) == 0:
            not_done_tasks = [task for task in self.tasks if task.state == STATE.NOT_DONE]
            if(len(not_done_tasks) != 0):
                return not_done_tasks[0]
        return None
            
    def update(self,time):
        # Met à jour toutes les tâches et renvoie les ressources qui ont été libérées
        res_to_free = []
        for t in self.tasks:
            res_to_free.append(t.update(time))
        return res_to_free
            
    def start(self,time):
        # Démarre la prochaine tâche
        t = next(task for task in self.tasks if task.state == STATE.NOT_DONE)
        if t != None:
            t.start(time)
            return t.ressource
        
    def is_done(self):
        # Renvoie true si le job est terminé
        return self.tasks[-1].is_done()
    
class InstanceHeuristic:
    # Classe d'une instance d'heuristique
    
    def __init__(self,instance_name,instance_path,info_path):
        
        # Récupération des différences paths
        instance_name = instance_name
        instances_path = instance_path
        instance_path = instances_path + instance_name  
        # Récupération des 
        with open(info_path, 'r') as f:
            instances_info = json.load(f)
            for i in instances_info:
                if i['name']==instance_name:
                    self.instance_info = i

        list_jobs = []
        n_machines = 0
        n_jobs = 0
        # Lecture de l'instance
        with open(instance_path,'r') as f:
            num_line = 0
            for line in f:
                if(line[0] != '#'):

                    re_line = re.sub(' +',';',line)
                    re_line = re.sub('\\n','',re_line)
                    split_line = re_line.split(';')
                    if (num_line == 0):
                        n_machines = int(split_line[1])
                        n_jobs = int(split_line[0])
                    else:
                        list_jobs.append(split_line)
                    num_line = num_line+1
        jobs = []
        for i in range(n_machines):
            job = JobForHeuristic(i)
            for j in range(0,2*n_jobs,2):
                job.add_task(TaskForHeuristic(int(list_jobs[i][j]),int(list_jobs[i][j+1]),i))
            jobs.append(job)
            
        self.n_machines = n_machines
        self.jobs=copy.deepcopy(jobs)
        self.machines_availability = [True]*n_machines
        self.time = 0
        self.order = np.array([]).astype(int)

    def get_max_tasks(self):
        nb_max=0
        for j in self.jobs:
            if(j.nb_task>nb_max):
                nb_max = j.nb_task
        return nb_max
    
    def check_possible_tasks(self):
        possible_tasks = []
        for j in self.jobs:
            next_task = j.get_next_task()
            if(next_task != None):
                if(self.machines_availability[next_task.ressource]):
                    possible_tasks = possible_tasks + [next_task]
        return possible_tasks
    
    def display_jobs(self):
        max_tasks_per_job = 0
        for j in self.jobs:
            if(max_tasks_per_job < len(j.tasks)):
                max_tasks_per_job = len(j.tasks)
        header=" "
        for t in range(max_tasks_per_job):
            header = header + " " + '{:^6}'.format("R D") + "  " 
        print(header)
        for j in self.jobs:
            print(j.to_string())
            
    def get_state_matrix(self):
        m = np.zeros((len(self.jobs),self.get_max_tasks()))
        for j_num,j in enumerate(self.jobs):
            for i,t in enumerate(j.tasks):
                if(t.state == STATE.ON_GOING):     
                    m[j_num][i] = 0             
                elif(t.state == STATE.DONE):
                    m[j_num][i] = 1           
                else:
                    m[j_num][i] = -1 
        return m
    
    def time_forward(self):
        self.time = self.time + 1
        for j in self.jobs:
            res_to_free = j.update(self.time)
            
            for r in res_to_free:
                if(r != -1):
                    self.machines_availability[r] = True
        
            
    def start_job(self,job_num):
        res_used = self.jobs[job_num].start(self.time)
        self.machines_availability[res_used] = False
        
    def is_done(self):
        done = True
        for j in self.jobs:
            done = done and j.is_done()
        return done
    
    def start_heuristic(self,fct=SPT,rand=0,display=False):
        if(display):
            matrix_state = self.get_state_matrix()
            fig, ax = plt.subplots()
            grid_x=[i+0.5 for i in range(matrix_state.shape[1])]
            ax.set_xticks(grid_x,minor=True)
            grid_y=[i+0.5 for i in range(matrix_state.shape[0])]
            ax.set_yticks(grid_y,minor=True)
            ax.grid(which='minor')
            ax.matshow(matrix_state, cmap='RdYlGn', vmin=-1, vmax=1)
            for i,jb in enumerate(self.jobs):
                for j,t in enumerate(jb.tasks):
                    text = ax.text(j, i, t.ressource,ha="center", va="center", color='black', fontsize=16)
            plt.xlabel('Tasks')
            plt.ylabel('Jobs')
            
        while not self.is_done():
            possible_tasks = self.check_possible_tasks()
            if(len(possible_tasks) > 0):
                if(random.random()<rand):
                    highest_prio_task = random.choice(possible_tasks)
                else:
                    highest_prio_task = fct(possible_tasks)
                self.start_job(highest_prio_task.job_num)
                self.order = np.append(self.order,highest_prio_task.job_num)
            else:
                self.time_forward()
                
                if(display):
                    matrix_state = self.get_state_matrix()
                    ax.matshow(matrix_state, cmap='RdYlGn', vmin=-1, vmax=1)
                    ax.set_title("TIME : "+str(self.time))
                    plt.pause(0.01)
  


In [ ]:
instance_name = 'ft06'
instances_path = 'JSPLIB/instances/'
instance_info_path = 'JSPLIB/instances.json'

instance = InstanceHeuristic(instance_name,instances_path,instance_info_path)
print("Nombre de machines : ",instance.n_machines)
print('Nombre de jobs : ',len(instance.jobs))
print('Taches : ')
instance.display_jobs()
print("Optimum : ",instance.instance_info['optimum'])

In [ ]:
instance.start_heuristic(fct=LPT,rand=0.1,display=False)
print(instance.order)

In [ ]:
print(instance.time)

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

class RepetitionOrder():
    def __init__(self,order,jobs,nb_machines):
        self.nb_machines = nb_machines
        self.order = copy.copy(order)
        self.duration = None
        self.jobs = copy.deepcopy(jobs)
        if(self.jobs[0].__class__ == JobForHeuristic):
            for j in self.jobs:
                j.__class__ = Job
                j.cast_task()
        self.jobs_end_time = [0] * len(self.jobs)
        self.ressources_end_time = [0] * nb_machines
        self.temp_var_1 = None
        self.temp_var_2 = None
        self.compute()
        
        
    def compute(self):
        job_task_index = [0] * len(self.jobs)
        for i in self.order:
            end_time, ressource_used = self.jobs[i].start(self.ressources_end_time,self.jobs_end_time
                                                            ,job_task_index[i])
            job_task_index[i] += 1
            self.update_end_times(ressource_used,i,end_time)
        self.duration = max(self.jobs_end_time)
   
    def display_gantt(self):
        colors = {}
        list_colors = ['dodgerblue','seagreen','sienna','turquoise','orchid','gold','black','white','red'
                       ,'blue','green','yellow','purple']
        fig = go.Figure(
            layout = {
                'barmode': 'stack',
                'xaxis': {'automargin': True},
                'yaxis': {'automargin': True}}
        )
        for j in self.jobs:
            for t in j.tasks:
                show_legend = False
                if(str(t.ressource) not in colors.keys()):
                    color_picked = list_colors[0]
                    list_colors.remove(color_picked)
                    colors[str(t.ressource)] =  color_picked
                    show_legend = True

                fig.add_bar(x=[t.duration],
                    y=['Job ' + str(j.num)],
                    base=t.start_time,
                    orientation='h',
                    showlegend=show_legend,
                    name=str(t.ressource),
                    marker={'color': colors[str(t.ressource)]}) 
        fig.update_layout(
            title="Gantt chart",
            xaxis_title="Time",
            yaxis_title="Jobs"
        )
        fig.update_xaxes(showgrid=True, gridwidth=1)
        fig.show()
        
    def get_score(self):
        return 1/self.duration

    def update_end_times(self,ressource_used,job_num,end_time):
        self.ressources_end_time[ressource_used] = end_time
        self.jobs_end_time[job_num] = end_time
            
    def get_swapped(self,ind1,ind2):
        res = self.order.copy()
        c = res[ind1]
        res[ind1] = res[ind2]
        res[ind2] = c
        return res
        
    def get_2swap_voisinage(self):
        voisins = []
        for i in range(len(self.order)):
            for j in range(i,len(self.order)):
                if(self.order[i] != self.order[j]):
                    voisins.append(RepetitionOrder(self.get_swapped(i,j),self.jobs,self.nb_machines))
                    
        for i in range(len(self.order)):
            for j in range(len(self.order)):
                order = copy.copy(self.order)
                poped = order[i]
                order = np.delete(order,i)
                order = np.insert(order,j,poped)
                voisins.append(RepetitionOrder(order,self.jobs,self.nb_machines))
        return voisins

def correct_order(order,jobs):
    jobs_sizes = [len(j.tasks) for j in jobs]
    jobs_tasks_counter = [0] * len(jobs_sizes)
    indexes_to_delete = []
    for i,e in enumerate(order):
        if(jobs_tasks_counter[e] == jobs_sizes[e]):
            indexes_to_delete.append(i)
        else:
            jobs_tasks_counter[e] += 1
            
    order = np.delete(order,indexes_to_delete)
    
    for i in range(len(jobs_sizes)):
        while jobs_tasks_counter[i] < jobs_sizes[i]:
            order = np.append(order,i)
            jobs_tasks_counter[i] += 1
    return order
        

def order_fusion(order_1,order_2,jobs):
    size_of_order = len(order_1)
    new_order = np.concatenate((order_1[0:size_of_order//2],order_2[size_of_order//2:-1]),axis=0)
    new_order = correct_order(new_order,jobs)
    return new_order
    
def mutate(order):
    mutation_done = False
    while not mutation_done:
        swap_1 = random.randint(0,len(order)-1)
        swap_2 = random.randint(0,len(order)-1)
        if(order[swap_1] != order[swap_2]):
            temp_var = order[swap_1]
            order[swap_1] = order[swap_2]
            order[swap_2] = temp_var
            mutation_done = True
    return order
    
# def get_concentration(rep_order,population,threshold):
#     Q = 0
#     for e in population:
#         if(np.count_nonzero(rep_order.order - e.order) > threshold):
#             Q += 1
#     return Q/len(population)

# def get_sum_concentration(population,threshold):
#     sum_concentration=0
#     for p in population:
#         sum_concentration += get_concentration(p,population,threshold)
#     return sum_concentration


# def get_sum_score(population):
#     sum_score = 0
#     for e in population:
#         sum_score += e.get_score()
#     return sum_score

# def get_fitness(rep_order, sum_score):
#     return rep_order.get_score()/sum_score



# def mixed_selection(population,nu,threshold):
#     new_population = []
#     sum_duration = get_sum_duration(population)
#     sum_concentration = get_sum_concentration(population,threshold)
#     for p in population:
#         # Fitness probability
#         pf = get_fitness(p,sum_duration)
#         # Concentration probability
#         pc = (1-get_concentration(p,population,threshold))/(len(population)-sum_concentration)
#         # Selection probability
#         ps = nu*pf + (1-nu) * pc 
#         if(random.random() < ps):
#             new_population.append(p)
#     return new_population

def get_crossover(base_population):
    population = copy.copy(base_population)
    pair_list = []
    new_list_of_order = []
    while len(population)>=2:
        element_1 = population.pop(random.randint(0,len(population)-1))
        element_2 = population.pop(random.randint(0,len(population)-1))
        pair_list.append([element_1,element_2])
        pair_list.append([element_2,element_1])
    for pair in pair_list:
        new_list_of_order.append(order_fusion(pair[0].order,pair[1].order,pair[0].jobs))
    return new_list_of_order

def get_mutated_orders(order_list,mutation_chance):
    for o in order_list:
        if(random.random() < mutation_chance):
            o = mutate(o)
    return order_list

def order_list_to_population(order_list,jobs,nb_machines):
    population = []
    for o in order_list:
        population.append(RepetitionOrder(o,jobs,nb_machines))
    return population

# def HGA(population,nu,threshold,mutation_proba):
#     crossovers = get_crossover(population)
#     mutated_crossovers = get_mutated_orders(crossovers,mutation_proba)
#     crossovers_population = order_list_to_population(mutated_crossovers,population[0].jobs,population[0].nb_machines)
#     crossovers_population_selection = mixed_selection(crossovers_population,nu,threshold)
#     return population + crossovers_population_selection

def get_best_subpopulation(population,size):
    population_copie = population
    new_population = []
    while len(new_population) != size:
        best_of_pop = get_best_voisin(population)
        population_copie.remove(best_of_pop)
        new_population.append(best_of_pop)
    return new_population
    
def get_best_voisin(population):
    if(len(population)>=1):
        best_voisin = population[0]
        best_perf = population[0].duration
        for v in population:
            if(v.duration <= best_perf):
                best_voisin = v
                best_perf = v.duration
        return best_voisin
    else: 
        return None
    
def get_mean_duration(population):
    mean_duration = 0
    for p in population:
        mean_duration += p.duration
    return mean_duration/len(population)

def get_best_fighters(rep_order_1,rep_order_2,comeback_chance=0,best_duration=None):
    if best_duration:
        if(rep_order_1.duration < best_duration):
            return rep_order_1
        if(rep_order_2.duration < best_duration):
            return rep_order_2
    if(rep_order_1.duration < rep_order_2.duration and random.random() > comeback_chance):
        return rep_order_1
    else:
        return rep_order_2

def FFA(population, size,comeback_chance,best_duration=None):
    new_population = []
    while len(new_population) != size:
        winner = get_best_fighters(random.choice(population),random.choice(population),comeback_chance,best_duration=best_duration)
        new_population.append(winner)
    return new_population
                             
def my_own_GA(population,size=None,mutation_chance=0.2,comeback_chance=0,best_duration=None):
    if not size :
        size=len(population)
    new_population = FFA(population,size=size,comeback_chance=comeback_chance,best_duration=best_duration) 
    children = get_crossover(new_population)
    children = get_mutated_orders(children,mutation_chance=mutation_chance)
    children = order_list_to_population(children,population[0].jobs,population[0].nb_machines)
    return children

In [ ]:
        
rep_order = RepetitionOrder(instance.order,instance.jobs,instance.n_machines) 
print('Repetition order from heuristic : ', rep_order.order)
print(rep_order.ressources_end_time)
print(rep_order.duration)
rep_order.display_gantt()

### Exploration du voisinage classique

In [ ]:
best_voisin = rep_order
nb_iteration = 10
for i in range(nb_iteration):
    print("Iteration : ",i)
    voisins = best_voisin.get_2swap_voisinage()
    best_voisin = get_best_voisin(voisins)
    print("Order : ",best_voisin.order)
    print("Duration : ", best_voisin.duration)
# best_voisin.display_gantt()
VOISIN_DIRECT = best_voisin

### Exploration du voisinage genetique

In [ ]:
best_voisin = rep_order
population = best_voisin.get_2swap_voisinage()
population = random.sample(population,100)

nb_iter = 100
new_population = population
start_mutation_chance = 0.2
mutation_chance_increase_rate = 1.001
mutation_chance = start_mutation_chance
for i in range(nb_iter):
    new_population = my_own_GA(new_population,mutation_chance=mutation_chance,best_duration=best_voisin.duration)
    best_of_pop = get_best_voisin(new_population)
    if(best_of_pop.duration < best_voisin.duration):
        best_voisin = best_of_pop
        mutation_chance = start_mutation_chance
        print("Best voisin : ",best_voisin.order)
        print("Temps : ",best_voisin.duration)
    else:
        mutation_chance = mutation_chance * mutation_chance_increase_rate
    if(i%10 == 0):print("Temps moyens de la population : ",get_mean_duration(new_population))
    if(best_voisin.duration == instance.instance_info['optimum']): break

### Analyse de l'exploration génétique

In [ ]:
nb_iter = 100
nb_iter_GA = 1000
population_size=100

mutation_chance = 0.2

comeback_chance = 0.2


comeback_chances = []
comeback_chances_to_plot = []
iteration_found = []
time_found = []


for i in range(nb_iter):
    found=False
#     rep_order = RepetitionOrder(instance.order,instance.jobs,instance.n_machines)
    rep_order = VOISIN_DIRECT
    best_voisin = rep_order
    population = best_voisin.get_2swap_voisinage()
    population = random.sample(population,population_size)
    new_population = population
    
    time_start = time.time()
    for j in range(nb_iter_GA):
        new_population = my_own_GA(new_population,
                                   mutation_chance=mutation_chance,
                                   comeback_chance=comeback_chance,
                                   best_duration=best_voisin.duration)
        best_of_pop = get_best_voisin(new_population)
        
        if(best_of_pop.duration < best_voisin.duration):
            best_voisin = best_of_pop
        
        if(best_voisin.duration == instance.instance_info['optimum']):
            found = True
            iteration_found.append(j)
            break

    time_end = time.time()
    time_found.append(time_end - time_start)
    if found: print("Iteration {}, optimum trouvé après {} itérations, Temps : {}, Solution : {}".format(i,j,time_end - time_start,best_voisin.duration))
    if not found: print("Iteration {}, optimum non trouvé, Temps : {}, Solution : {}".format(i,time_end - time_start,best_voisin.duration))


### Fusion exploration directe + GA

In [ ]:

instance = InstanceHeuristic('ft10',instances_path,instance_info_path)
print('Début heuristique')
instance.start_heuristic(fct=LPT,rand=0.1,display=False)
rep_order = RepetitionOrder(instance.order,instance.jobs,instance.n_machines) 
print("Descente directe : ",rep_order.order)
print("Durée : ",rep_order.duration)
print("Fin heuristique")

best_voisin = rep_order
nb_iteration = 1000
print("Début descente directe")
for i in range(nb_iteration):
    voisins = best_voisin.get_2swap_voisinage()
    if((best_voisin.order == get_best_voisin(voisins).order).all()):
        print("Convergence terminée")
        break
    else:
        best_voisin = get_best_voisin(voisins)
        print('Meilleur temps trouvé :',best_voisin.duration)

print("Descente directe : ",best_voisin.order)
print("Durée : ",best_voisin.duration)

print("Début exploration génétique")
optimum = instance.instance_info['optimum']
nb_iter = 100
nb_iter_GA = 10000
population_size=100

mutation_chance_max = 0.5
mutation_chance_min= 0.1
mutation_chance = 0.2
mutation_increase_rate_start = 1.01
mutation_decrease_rate = 0.999
mutation_increase_rate = mutation_increase_rate_start


comeback_chance_max = 0.5
comeback_chance_min= 0.2
comeback_chance = 0.2
comeback_increase_rate_start = 1.005
comeback_increase_rate = comeback_increase_rate_start


optimum_found=False

time_start = time.time()

population = best_voisin.get_2swap_voisinage()
population = get_best_subpopulation(population,population_size)
new_population = population
    
for j in range(nb_iter_GA):
    new_population = my_own_GA(new_population,
                               size=population_size,
                               mutation_chance=mutation_chance,
                               comeback_chance=comeback_chance,
                               best_duration=best_voisin.duration)
    best_of_pop = get_best_voisin(new_population)
    
    if(comeback_chance>comeback_chance_max or comeback_chance<comeback_chance_min):
        comeback_increase_rate = 2-comeback_increase_rate
        
    if(mutation_chance>mutation_chance_max or mutation_chance<mutation_chance_min):
        if(mutation_chance>mutation_chance_max):
            mutation_increase_rate = mutation_decrease_rate
        else:
            mutation_increase_rate = mutation_increase_rate_start
            
    if(best_of_pop.duration < best_voisin.duration):
        print('Meilleur temps trouvé :',best_of_pop.duration)
        comeback_increase_rate = comeback_increase_rate_start
        comeback_chance = comeback_chance_min
        mutation_increase_rate = mutation_increase_rate_start
        mutation_chance = mutation_chance_min
        best_voisin = best_of_pop
        
        print("Début descente directe")
        for i in range(nb_iteration):
            voisins = best_voisin.get_2swap_voisinage()
            if((best_voisin.order == get_best_voisin(voisins).order).all()):
                print("Convergence terminée")
                break
            else:
                best_voisin = get_best_voisin(voisins)
                print('Meilleur temps trouvé :',best_voisin.duration)
        
        print("Descente directe : ",best_voisin.order)
        print("Durée : ",best_voisin.duration)
        population = best_voisin.get_2swap_voisinage()
        population = get_best_subpopulation(population,population_size)
        new_population = population
    else:
        comeback_chance *= comeback_increase_rate
        mutation_chance *= mutation_increase_rate

    if(best_voisin.duration == optimum):
        optimum_found = True
        break
    if(j%50==0):
        mean_duration = get_mean_duration(new_population)
        print("Iteration : {}, moyenne des durée : {}".format(j,mean_duration))

time_end = time.time()
if(optimum_found): print("Optimum trouvé après {} itérations".format(j))
else: print("Optimum non trouvé après {} itérations".format(j))
print("Temps : ", time_end-time_start)
print("Durée final", best_voisin.duration)
print("Ordre final", best_voisin.order)
best_voisin.display_gantt()

In [ ]:
instance.instance_info

In [ ]:

a = [1,2,3,4]
new_a = np.delete(a, 1)

In [ ]:
np.insert(new_a,1,2)

In [ ]:
get_mean_duration(new_population)